In [87]:
print("Enter keyword you'd like to crawl")
keyword = input()

Enter keyword you'd like to crawl
대구맛집


In [88]:
import requests
from urllib.request import Request
import urllib.parse as parse
import urllib.request as request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains as AC
import json
import time
import pandas as pd

In [89]:
def getNaverURLs(keyword, driver):
    page=1
    naverURLs = list()
    idDict = dict()
    latitudes = list()
    longtitudes = list()
    addresses = list()
    names = list()
    
    while(True):
        url = ('https://map.naver.com/v5/api/search?caller=pcweb&query='+keyword+
               '&type=restaurant&searchCoord=128.4675979614258;35.871525056258626&page='+str(page)+
               '&displayCount=10&isPlaceRecommendationReplace=true&lang=ko')
#         print('url-->')
#         print(url)
        driver.get(url)
        
        #for page in count(1):
        response = driver.page_source
        soup = BeautifulSoup(response,'html.parser')
#         print('response-->')
#         print(soup.text)
        
        JSON=json.loads(soup.text)
        if(JSON.get('error')==None):
            print('Reading page',page)
            searchResult = JSON.get('result').get('place').get('list')
            for item in searchResult:
                itemID = item.get('id')
#                 name = item.get('name')
                longtitude = item.get('x')
                latitude = item.get('y')
                address = item.get('address')
                name = item.get('name')
                
                if not(itemID in idDict):
                    reviewURL = 'https://pcmap.place.naver.com/restaurant/'+itemID+'/review/visitor?entry=pll&from=map'
                    naverURLs.append(reviewURL)
                    idDict[itemID] = True
                    longtitudes.append(longtitude)
                    latitudes.append(latitude)
                    addresses.append(address)
                    names.append(name)
            page = page + 1
            
        else:
            break
    return naverURLs, latitudes, longtitudes, addresses, names

In [90]:
import pandas as pd

def getlink(url, longtitude, latitude, address, name, driver):
    driver.get(url)
    
    df = pd.DataFrame(columns=['date','reviews','stars', 'category', 'address','lat', 'lng','name'])
    
    #clickCnt=0
    count = 0 
    while count<80:
        try: 
            a = driver.find_element_by_css_selector("a._3iTUo")
            if a.get_attribute('href')[:-1]!=driver.current_url:
                print('except: no 더보기 button')
                return df
            a.click()
            count=count+1
            
            time.sleep(1)
        except:
            break
    try:
        stars = driver.find_elements_by_xpath("//span[@class='_2tObC']")
        reviews = driver.find_elements_by_xpath("//span[@class='WoYOw']")
        date_div = driver.find_elements_by_xpath("//div[@class='ZvQ8X']")
        date = [item.find_element_by_xpath(".//span[@class='_3WqoL']") for item in date_div]
        category = driver.find_element_by_xpath("//span[@class='_3ocDE']")
        #name = driver.find_element_by_xpath("//span[@class='_3XamX']")
        #print(name.text)
        datas = driver.find_elements_by_xpath("//div[@class='_1Z_GL']")
    except:
        return df
        
#         date = [] 
#         for i, item in enumerate(date_raw):
#             if i%3==0:
#                 date.append(item)
                
#         print(len(date), len(stars))

    reviews_str = list()
    stars_str = list()
    date_str = list()
    
    for i in range(0, len(reviews)):
        reviews_str.append(reviews[i].text)
    for i in range(0, len(stars)):
        stars_str.append(stars[i].text)
    for i in range(0, len(date)):
        date_str.append(date[i].text)
    
    for i in range(0, len(datas)):
        if datas[i].get_attribute('innerHTML').find('WoYOw')==-1:
            reviews_str.insert(i, "")
#             if datas[i].get_attribute('innerHTML').find('_2tObC')==-1:
#                 stars_str.insert(i, "")
    
    #df['names']=name.text
    df['date']=date_str
    
    df['reviews']=reviews_str
    
    df['stars']=stars_str
    df['category']=category.text
    df['address']=address
    df['lat']=latitude
    df['lng']=longtitude
    df['name'] = name
    return df

In [91]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)
naverURLs, addresses, latitudes, longtitudes, names = getNaverURLs(keyword, driver)

Reading page 1
Reading page 2
Reading page 3
Reading page 4
Reading page 5
Reading page 6
Reading page 7
Reading page 8
Reading page 9
Reading page 10
Reading page 11
Reading page 12
Reading page 13
Reading page 14
Reading page 15
Reading page 16
Reading page 17
Reading page 18
Reading page 19
Reading page 20
Reading page 21
Reading page 22
Reading page 23
Reading page 24
Reading page 25
Reading page 26
Reading page 27
Reading page 28
Reading page 29
Reading page 30
Reading page 31


In [92]:
len(naverURLs), len(addresses), len(latitudes), len(longtitudes), len(names)

(310, 310, 310, 310, 310)

In [93]:
naverURLs

['https://pcmap.place.naver.com/restaurant/13571958/review/visitor?entry=pll&from=map',
 'https://pcmap.place.naver.com/restaurant/36625257/review/visitor?entry=pll&from=map',
 'https://pcmap.place.naver.com/restaurant/20601192/review/visitor?entry=pll&from=map',
 'https://pcmap.place.naver.com/restaurant/13571722/review/visitor?entry=pll&from=map',
 'https://pcmap.place.naver.com/restaurant/11848917/review/visitor?entry=pll&from=map',
 'https://pcmap.place.naver.com/restaurant/11850601/review/visitor?entry=pll&from=map',
 'https://pcmap.place.naver.com/restaurant/12428384/review/visitor?entry=pll&from=map',
 'https://pcmap.place.naver.com/restaurant/13495269/review/visitor?entry=pll&from=map',
 'https://pcmap.place.naver.com/restaurant/30943631/review/visitor?entry=pll&from=map',
 'https://pcmap.place.naver.com/restaurant/13572427/review/visitor?entry=pll&from=map',
 'https://pcmap.place.naver.com/restaurant/35644433/review/visitor?entry=pll&from=map',
 'https://pcmap.place.naver.com/

In [ ]:
df = getlink(naverURLs[1], addresses[1], latitudes[1], longtitudes[1], names[1], driver)

In [86]:
df

,date,reviews,stars,category,address,lat,lng,name
0,2020.12.17,좋아요,3.5,호텔,대구광역시 중구 동산동 360,128.5860059,35.8664277,엘디스 리젠트 호텔
1,2020.12.17,좋아요,4,호텔,대구광역시 중구 동산동 360,128.5860059,35.8664277,엘디스 리젠트 호텔
2,2020.12.17,굿,4.5,호텔,대구광역시 중구 동산동 360,128.5860059,35.8664277,엘디스 리젠트 호텔
3,2020.11.26,"위생상태 좋은편이고, 호텔은 항상 건조한데 가습기 대여가능해서 좋았어요. 체크인할때...",3.5,호텔,대구광역시 중구 동산동 360,128.5860059,35.8664277,엘디스 리젠트 호텔
4,2020.11.25,커피맛은 쏘쏘 직원 완전 불친절 호텔 1층 탐탐인데 직원이 왜 이렇게 블퉁시럽지? ㅡㅡ,2.5,호텔,대구광역시 중구 동산동 360,128.5860059,35.8664277,엘디스 리젠트 호텔
5,2020.11.24,겨울에 방이 너무 추워요...... 중앙 난방인데 제대로 안 돼서 정말 추워서 혼났...,3,호텔,대구광역시 중구 동산동 360,128.5860059,35.8664277,엘디스 리젠트 호텔
6,2020.11.21,호텔안에 편의점이 편리합니다,3,호텔,대구광역시 중구 동산동 360,128.5860059,35.8664277,엘디스 리젠트 호텔
7,2020.11.21,호텔 내 커피숍인데 좀 느리시네요,2,호텔,대구광역시 중구 동산동 360,128.5860059,35.8664277,엘디스 리젠트 호텔
8,2020.11.14,뷰가 좋음 자동블라인드 좋아요,5,호텔,대구광역시 중구 동산동 360,128.5860059,35.8664277,엘디스 리젠트 호텔
9,2020.11.11,굿,4,호텔,대구광역시 중구 동산동 360,128.5860059,35.8664277,엘디스 리젠트 호텔


In [58]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)
df = pd.DataFrame(columns=['date','reviews','stars', 'category','location', 'address','lat', 'lng'])
for i in range(0, len(naverURLs)):
    df = pd.concat([df, getlink(naverURLs[i], longtitudes[i], latitudes[i], addresses[i], driver)], ignore_index = True)

이월드
수성못상화동산
해맞이공원
동제미술관
두류공원
앞산
옥연지 송해공원
스파밸리
83타워
네이처파크
대구아쿠아리움
반고개무침회골목
앞산공원전망대
팔공산케이블카
월광수변공원
동촌유원지
대새목장
이월드코코몽눈빛마을
국채보상운동기념공원
에코테마파크 대구숲
대구수목원
EXCO
두류공원 두류야구장
달성공원
공룡공원
디아크문화관
마비정벽화마을
스파크랜드
국립대구과학관
진골목
아이니테마파크
안지랑 곱창 거리
앞산해넘이전망대
미성온천
봉무공원
대견사지
안지랑골체육공원
비슬산자연휴양림
수성유원지
침산공원
앞산케이블카
화원유원지
원조닭똥집골목
동성로
운암지 수변공원
대구미술관
불로동고분군
율하체육공원
최정산
앞산공원


In [59]:
df.to_csv("대구-명소.csv", index=False, encoding='utf-8-sig')